In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# Initialize LLM
llm = ChatOpenAI(temperature=0.1)

# Define the cache directory
cache_dir = LocalFileStore("./.cache")

# Initialize the text splitter
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=300,
    chunk_overlap=30,
)

# Load and split the document
loader = UnstructuredFileLoader("./files/BAU_Report_May.docx")
docs = loader.load_and_split(text_splitter=splitter)

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize cache-backed embeddings
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# Initialize the vector store using the cached embeddings
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you dont know the answer, just say you don't know, don't make it up:\n\n{context}",),
    ("human", "{question}")
])

# Initialize the retrieval chain
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt | llm

# Run the chain with a query
chain.invoke("Describe 1 acheivement")




c:\Users\Jonathan\Documents\fullstack-gpt\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\Jonathan\Documents\fullstack-gpt\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


AIMessage(content='One achievement highlighted in the report is the completion of the Canvas access improvements by the Resource Team.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 1123, 'total_tokens': 1142}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8781ae26-79a4-4440-a36f-4e3254bc6553-0')